# Рекомендательные системы

- Целевая метрика precision@5 >=0.2
- Бейзлайн решения - MainRecommender
- На github должен быть файл recommendations.csv (user_id | [rec_1, rec_2, ...] с рекомендациями. rec_i - реальные id item-ов (из retail_train.csv)

### Подключение библиотек

In [80]:
import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

import lightgbm as lgb
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# Для работы с матрицами
from scipy.sparse import csr_matrix
from implicit import als

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import pre_filter_items
#from src.recommenders import MainRecommender

import warnings
warnings.filterwarnings('ignore')

### Загрузка данных

In [81]:
data = pd.read_csv('./data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [82]:
test = pd.read_csv('./data/retail_test1.csv')
test.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,1340,41652823310,664,912987,1,8.49,446,0.0,52,96,0.0,0.0
1,588,41652838477,664,1024426,1,6.29,388,0.0,8,96,0.0,0.0


In [83]:
item_features = pd.read_csv('./data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

In [84]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [85]:
user_features = pd.read_csv('./data/hh_demographic.csv')
user_features.columns = [col.lower() for col in user_features.columns]
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

In [86]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


### Схема обучения и валидации:
* давние покупки | 6 недель | 3 недели 

In [87]:
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

In [88]:
users_lvl_1 = data_train_lvl_1.user_id.unique()
users_lvl_2 = data_val_lvl_1.user_id.unique()
users_lvl_3 = data_val_lvl_2.user_id.unique()

new_users_lvl_2 = list(set(users_lvl_2) - set(users_lvl_1))
new_users_lvl_3 = list(set(users_lvl_3) - (set(users_lvl_1) | set(users_lvl_2)))

add_to_lvl_2 = list(set(users_lvl_3) - (set(users_lvl_2)))

new_users_lvl_2, new_users_lvl_3, len(add_to_lvl_2)

([1984], [], 126)

#### Уменьшение размерности до top 5000

In [89]:
n_items_before = data_train_lvl_1['item_id'].nunique()
data_train_lvl_1 = pre_filter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)
n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


### Генерация  признаков для модели

#### Пользовательские признаки

In [90]:
user_features['age_desc'].replace(
    {'19-24': 22, '25-34': 30, '35-44': 40, '45-54': 50, '55-64': 60, '65+': 70},
    inplace=True)

user_features['marital_status_code'].replace(
    {'U': 0, 'A': 1, 'B': 2}, inplace=True)

user_features['income_desc'].replace(
    {'Under 15K': 10, '15-24K': 20, '25-34K':30, '35-49K': 40,
     '50-74K': 62, '75-99K': 87, '100-124K': 112, '125-149K': 137, 
     '150-174K': 162, '175-199K': 187, '200-249K': 225, '250K+':275}, inplace=True)

user_features['homeowner_desc'].replace(
    {'Unknown': 0, 'Probable Renter': 1, 'Renter': 2,
     'Probable Owner': 3, 'Homeowner': 4}, inplace=True)

user_features['hh_comp_desc'].replace(
    {'Unknown': 0, 'Single Male': 1, 'Single Female': 2,
     '1 Adult Kids': 3, '2 Adults No Kids': 4, '2 Adults Kids':5},inplace=True)

user_features['household_size_desc'].replace({'5+': 5}, inplace=True) 

user_features['kid_category_desc'].replace(
    {'None/Unknown': 0, '3+': 3}, inplace=True)

user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,70,1,40,4,4,2,0,1
1,50,1,62,4,4,2,0,7


#### Товарные признаки

In [91]:
names = ['manufacturer', 'department', 'commodity_desc', 'sub_commodity_desc', 'curr_size_of_product']
for name in names:
    new_name = name + '_freq'
    a = item_features[name].value_counts()
    ind = a.index.tolist()
    for i in ind:
        item_features.loc[item_features[name] == i, new_name] = a[i]

item_features['brand'] = np.where(item_features['brand']=='Private', 0, 1)

commodities = item_features.commodity_desc.value_counts()
commodities_list = commodities.keys().tolist()
for i, name in enumerate(commodities_list):
    item_features.loc[item_features['commodity_desc'] == name, 'commodity_category'] = i

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,manufacturer_freq,department_freq,commodity_desc_freq,sub_commodity_desc_freq,curr_size_of_product_freq,commodity_category
0,25671,2,GROCERY,1,FRZN ICE,ICE - CRUSHED/CUBED,22 LB,1411.0,39021.0,29.0,29.0,12.0,270.0
1,26081,2,MISC. TRANS.,1,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,,1411.0,490.0,490.0,429.0,30607.0,56.0


In [92]:
def get_user_item_features(data_train_lvl_1):
    # час совершения транзакции
    X = data_train_lvl_1.copy()
    X['hour'] = X['trans_time'] // 100
    user_item_features = X.groupby(['user_id', 'item_id'])['hour'].median().reset_index()
    user_item_features.columns = ['user_id', 'item_id', 'median_sales_hour']
    
    # день недели совершения транзакции
    X['weekday'] = X['day'] % 7
    df = X.groupby(['user_id', 'item_id'])['weekday'].median().reset_index()
    df.columns = ['user_id', 'item_id', 'median_weekday']
    user_item_features = user_item_features.merge(df, on=['user_id', 'item_id'])
    
    # cреднее кол-во дней между покупками
    df = X.groupby('user_id')['day'].nunique().reset_index()
    df['mean_visits_interval'] = (X.groupby('user_id')['day'].max() - X.groupby('user_id')['day'].min()) / df['day']
    user_item_features = user_item_features.merge(df[['user_id', 'mean_visits_interval']], on=['user_id'])
    
    # средний чек корзины клиента
    df = X.groupby(['user_id', 'basket_id'])['sales_value'].sum().reset_index()
    df = df.groupby('user_id')['sales_value'].mean().reset_index()
    df.columns = ['user_id', 'mean_check']
    user_item_features = user_item_features.merge(df, on=['user_id'])
    
    # кол-во магазинов, в которых продавался товар
    df = X.groupby(['item_id'])['store_id'].nunique().reset_index()
    df.columns = ['item_id', 'n_stores']
    user_item_features = user_item_features.merge(df, on=['item_id'])
    
    # кол-во уникальных товаров, купленных клиентом
    df = X.groupby(['user_id'])['item_id'].nunique().reset_index()
    df.columns = ['user_id', 'n_items']
    user_item_features = user_item_features.merge(df, on=['user_id'])
    
    # кол-во транзакций клиента
    df = X.groupby(['user_id'])['item_id'].count().reset_index()
    df.columns = ['user_id', 'n_transactions']
    user_item_features = user_item_features.merge(df, on=['user_id'])
    
    # mean / max / std кол-ва уникальных товаров в корзине клиента
    df = X.groupby(['user_id', 'basket_id'])['item_id'].nunique().reset_index()
    df1 = df.groupby('user_id')['item_id'].mean().reset_index()
    df1.columns = ['user_id', 'mean_n_items_basket']
    user_item_features = user_item_features.merge(df1, on=['user_id'])

    df2 = df.groupby('user_id')['item_id'].max().reset_index()
    df2.columns = ['user_id', 'max_n_items_basket']
    user_item_features = user_item_features.merge(df2, on=['user_id'])

    df3 = df.groupby('user_id')['item_id'].std().reset_index()
    df3.columns = ['user_id', 'std_n_items_basket']
    user_item_features = user_item_features.merge(df3, on=['user_id'])
    
    # mean / max / std кол-ва уникальных категорий в корзине клиента
    X = X.merge(item_features[['item_id', 'commodity_desc']], on=['item_id'])
    df = X.groupby(['user_id', 'basket_id'])['commodity_desc'].nunique().reset_index()
    df1 = df.groupby('user_id')['commodity_desc'].mean().reset_index()
    df1.columns = ['user_id', 'mean_n_item_categories_basket']
    user_item_features = user_item_features.merge(df1, on=['user_id'])

    df2 = df.groupby('user_id')['commodity_desc'].max().reset_index()
    df2.columns = ['user_id', 'max_n_item_categories_basket']
    user_item_features = user_item_features.merge(df2, on=['user_id'])

    df3 = df.groupby('user_id')['commodity_desc'].std().reset_index()
    df3.columns = ['user_id', 'std_n_item_categories_basket']
    user_item_features = user_item_features.merge(df3, on=['user_id'])
    
    # эмбеддинги товаров
    recommender = MainRecommender(X)
    df = recommender.model.item_factors
    n_factors = recommender.model.factors
    ind = list(recommender.id_to_itemid.values())
    df = pd.DataFrame(df, index=ind).reset_index()
    df.columns = ['item_id'] + ['factor_' + str(i + 1) for i in range(n_factors)]
    user_item_features = user_item_features.merge(df, on=['item_id'])
    
    # эмбеддинги пользователей
    df = recommender.model.user_factors
    ind = list(recommender.id_to_userid.values())
    df = pd.DataFrame(df, index=ind).reset_index()
    df.columns = ['user_id'] + ['user_factor_' + str(i + 1) for i in range(n_factors)]
    user_item_features = user_item_features.merge(df, on=['user_id'])
    
    return user_item_features

In [93]:
user_item_features = get_user_item_features(data_train_lvl_1)
user_item_features.head(2)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

,user_id,item_id,median_sales_hour,median_weekday,mean_visits_interval,mean_check,n_stores,n_items,n_transactions,mean_n_items_basket,...,user_factor_11,user_factor_12,user_factor_13,user_factor_14,user_factor_15,user_factor_16,user_factor_17,user_factor_18,user_factor_19,user_factor_20
0,1,823721,13.0,4.0,NaN,51.962097,94,220,1263,12.629032,...,3.21866,2.24128,2.27597,-1.7781,1.073132,0.912612,2.079721,1.307954,-1.855253,11.981043
1,1,823990,15.0,6.0,NaN,51.962097,96,220,1263,12.629032,...,3.21866,2.24128,2.27597,-1.7781,1.073132,0.912612,2.079721,1.307954,-1.855253,11.981043


In [94]:
def get_candidates(data_train_lvl_1, data_train_lvl_2, N, add_to_lvl_2):
    recommender = MainRecommender(data_train_lvl_1)

    users_lvl_1 = data_train_lvl_1['user_id'].unique()
    users_lvl_2 = data_train_lvl_2['user_id'].unique().tolist()
    if add_to_lvl_2:
        users_lvl_2 += add_to_lvl_2

    current_users = list(set(users_lvl_2) & set(users_lvl_1))    
    new_users = list(set(users_lvl_2) - set(users_lvl_1))

    df = pd.DataFrame(users_lvl_2, columns=['user_id'])
    cond_1 = df['user_id'].isin(current_users)
    df.loc[cond_1, 'candidates'] = df.loc[cond_1, 'user_id'].apply(
        lambda x: recommender.get_own_recommendations(x, N))

    if new_users:
        cond_2 = df['user_id'].isin(new_users)
        df.loc[cond_2, 'candidates'] = df.loc[cond_2, 'user_id'].apply(
            lambda x: recommender.overall_top_purchases[:N])
        
    return df

In [95]:
def get_targets_lvl_2(data_train_lvl_1, data_train_lvl_2, user_item_features, N, add_to_lvl_2=None):
    
    users_lvl_2 = get_candidates(data_train_lvl_1, data_train_lvl_2, N, add_to_lvl_2)
    
    df = pd.DataFrame({'user_id': users_lvl_2['user_id'].values.repeat(N),
                       'item_id': np.concatenate(users_lvl_2['candidates'].values)})

    targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
    targets_lvl_2['target'] = 1 

    targets_lvl_2 = df.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')
    targets_lvl_2['target'].fillna(0, inplace= True)
    
    targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
    targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')
    targets_lvl_2 = targets_lvl_2.merge(
        user_item_features, on=['user_id', 'item_id'], how='left')
    
    return targets_lvl_2

In [96]:
N = 500
targets_lvl_2 = get_targets_lvl_2(data_train_lvl_1, data_train_lvl_2, user_item_features, N, add_to_lvl_2)

print(f'число пользователей: {targets_lvl_2.user_id.nunique()}')
print(f'среднее число покупок: {round(targets_lvl_2["target"].mean(), 4)}')

targets_lvl_2.head(2)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

число пользователей: 2280
среднее число покупок: 0.034


,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,manufacturer_freq,...,user_factor_11,user_factor_12,user_factor_13,user_factor_14,user_factor_15,user_factor_16,user_factor_17,user_factor_18,user_factor_19,user_factor_20
0,2070,999999,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2070,1082185,1.0,2.0,PRODUCE,1.0,TROPICAL FRUIT,BANANAS,40 LB,1411.0,...,-0.869849,2.528729,0.963255,10.962255,0.145305,-0.032533,0.349288,-1.793689,1.692278,1.084573


### Отбор признаков

In [97]:
SELECTED_FEATURES_NAMES = ['brand',
                           'manufacturer_freq', 'department_freq', 'commodity_desc_freq',
                           'sub_commodity_desc_freq', 'curr_size_of_product_freq',
                           'commodity_category',
                           'age_desc', 'marital_status_code', 'income_desc', 
                           'homeowner_desc', 'hh_comp_desc', 'manufacturer', 
                           'median_sales_hour', 'median_weekday',
                           'mean_check', 
                           'n_stores', 'n_items', 'n_transactions', 
                           'mean_n_items_basket', 'max_n_items_basket', 'std_n_items_basket',
                           'mean_n_item_categories_basket', 'max_n_item_categories_basket', 
                           'std_n_item_categories_basket',
                           'factor_1', 'factor_2', 'factor_3', 'factor_4', 'factor_5',
                           'factor_6', 'factor_7', 'factor_8', 'factor_9', 'factor_10',
                           'factor_11', 'factor_12', 'factor_13', 'factor_14', 'factor_15',
                           'factor_16', 'factor_17', 'factor_18', 'factor_19', 'factor_20',          
                           'user_factor_1', 'user_factor_2', 'user_factor_3', 'user_factor_4',
                           'user_factor_5', 'user_factor_6', 'user_factor_7', 'user_factor_8',
                           'user_factor_9', 'user_factor_10', 'user_factor_11', 'user_factor_12',
                           'user_factor_13', 'user_factor_14', 'user_factor_15', 'user_factor_16',
                           'user_factor_17', 'user_factor_18', 'user_factor_19', 'user_factor_20',                           
                          ]
categorical = ['marital_status_code','homeowner_desc', 
               'hh_comp_desc', 'manufacturer',
               'commodity_category'
              ]

In [98]:
SELECTED_FEATURES_NAMES_cb = [i for i in SELECTED_FEATURES_NAMES if not i in categorical]

In [99]:
def run_model_cb(targets_lvl_2):    
    X_train, X_valid, y_train, y_valid = train_test_split(targets_lvl_2[SELECTED_FEATURES_NAMES_cb].fillna(0),
                                                          targets_lvl_2[['target']],
                                                          test_size=0.2, random_state=16,
                                                          stratify=targets_lvl_2[['target']])

    dtrain = Pool(data=X_train, label=y_train) #, cat_features=categorical
    dvalid = Pool(data=X_valid, label=y_valid) #, cat_features=categorical

    params_cb = {"n_estimators":5000,
                 "learning_rate": 0.1,
                 "loss_function": "Logloss",
                 "eval_metric": "AUC",
                 "task_type": "CPU",
                 "max_bin": 30,
                 "early_stopping_rounds": 30,
                 "verbose": 1000,
                 "max_depth": 10,
                 "l2_leaf_reg": 80,
                 "thread_count": 6,
                 "random_seed": 51} 

    model_cb = CatBoostClassifier(**params_cb)
    model_cb.fit(dtrain, eval_set=[dvalid])

    
    return model_cb

In [100]:
def run_model_lgb(targets_lvl_2):
    X_train, X_valid, y_train, y_valid = train_test_split(targets_lvl_2[SELECTED_FEATURES_NAMES].fillna(0),
                                                          targets_lvl_2[['target']],
                                                          test_size=0.2, random_state=16,
                                                          stratify=targets_lvl_2[['target']])

    dtrain = lgb.Dataset(X_train, y_train, categorical_feature=categorical)
    dvalid = lgb.Dataset(X_valid, y_valid, categorical_feature=categorical)

    params_lgb = {"boosting_type": "gbdt",
                  "objective": "binary", 
                  "metric": "auc",
                  "num_boost_round": 10000,
                  "learning_rate": 0.1,
                  "class_weight": 'balanced',
                  "max_depth": 10,
                  "n_estimators": 5000,
                  "n_jobs": 6,
                  "seed": 12} 

    model_lgb = lgb.train(params=params_lgb,
                          train_set=dtrain,  
                          valid_sets=[dtrain, dvalid],
                          categorical_feature=categorical,
                          verbose_eval=1000,
                          early_stopping_rounds=30)
    
    return model_lgb

In [101]:
model_lgb = run_model_lgb(targets_lvl_2)
# [2426]	training's auc: 0.964429	valid_1's auc: 0.896633

[LightGBM] [Warning] Unknown parameter: class_weight
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: class_weight
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 31255, number of negative: 888853
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.080952 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12754
[LightGBM] [Info] Number of data points in the train set: 920108, number of used features: 65
[LightGBM] [Warning] Unknown parameter: class_weight
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] [bi

In [102]:
model_cb = run_model_cb(targets_lvl_2)
# bestTest = 0.9069283804
# bestIteration = 2598

0:	test: 0.8339141	best: 0.8339141 (0)	total: 338ms	remaining: 28m 10s
1000:	test: 0.8851990	best: 0.8852002 (999)	total: 4m 52s	remaining: 19m 28s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.8852317311
bestIteration = 1014

Shrink model to first 1015 iterations.


In [103]:
predictions_lgb_train = model_lgb.predict(targets_lvl_2[SELECTED_FEATURES_NAMES].fillna(0))
predictions_cb_train = model_cb.predict_proba(targets_lvl_2[SELECTED_FEATURES_NAMES_cb].fillna(0))[:, 1]

In [104]:
preds_train = pd.DataFrame(zip(predictions_lgb_train, 
                               predictions_cb_train),
                           columns=['lgb', 'cb']).mean(axis=1).values
roc_auc_score(targets_lvl_2['target'], preds_train)
# 0.9557067627903868

0.9080166212914565

### Обработка

In [105]:
def get_predictions(targets_lvl_2, raw_predictions, prefix='lgb'): 
    df = targets_lvl_2[['user_id', 'item_id']]
    df['predictions'] = raw_predictions

    df = df.groupby(['user_id', 'item_id'])['predictions'].median().reset_index()
    df = df.sort_values(['predictions'], ascending=False).groupby(['user_id']).head(5)

    df = df.groupby('user_id')['item_id'].unique().reset_index()
    df.columns = ['user_id', prefix + '_recommendations']
    
    return df

In [106]:
def get_results(data_val_lvl_2, targets_lvl_2, preds_lgb, preds_cb, combined_preds):
    result = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
    result.columns=['user_id', 'actual']
    
    prefixes = ['lgb', 'cb', 'cb_lgb']
    predictions = [preds_lgb, preds_cb, combined_preds]
    
    for i, preds in enumerate(predictions):
        df = get_predictions(targets_lvl_2, preds, prefixes[i])
        result = result.merge(df, on='user_id', how='left')

    return result

In [107]:
result_lvl_2 = get_results(data_val_lvl_2, targets_lvl_2, 
                           predictions_lgb_train,
                           predictions_cb_train, 
                           preds_train)
result_lvl_2.head(2)

,user_id,actual,lgb_recommendations,cb_recommendations,cb_lgb_recommendations
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[940947, 1082185, 872137, 1082212, 1029743]","[1082185, 872137, 940947, 840361, 865456]","[1082185, 940947, 872137, 840361, 1082212]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 1133018, 1053690, 951590, 916381]","[1101010, 1106523, 1076875, 951590, 5569230]","[1106523, 1101010, 951590, 1053690, 916381]"


### Метрики

In [108]:
result_lvl_2.apply(lambda row: precision_at_k(row['lgb_recommendations'], row['actual'], 5), axis=1).mean()

0.320666013712047

In [109]:
result_lvl_2.apply(lambda row: precision_at_k(row['cb_recommendations'], row['actual'], 5), axis=1).mean()

0.3420176297747309

In [110]:
result_lvl_2.apply(lambda row: precision_at_k(row['cb_lgb_recommendations'], row['actual'], 5), axis=1).mean()

0.3432908912830562

### Построение прогнозов

In [111]:
validation_weeks = 6
data_train = data[data['week_no'] < data['week_no'].max() - validation_weeks]
data_valid = data[data['week_no'] >= data['week_no'].max() - validation_weeks]

In [112]:
users_lvl_1 = data_train.user_id.unique()
users_lvl_2 = data_valid.user_id.unique()
users_lvl_3 = test.user_id.unique()

new_users_lvl_2 = list(set(users_lvl_2) - set(users_lvl_1))
new_users_lvl_3 = list(set(users_lvl_3) - (set(users_lvl_1) | set(users_lvl_2)))

add_to_lvl_2 = list(set(users_lvl_3) - (set(users_lvl_2)))

new_users_lvl_2, new_users_lvl_3, len(add_to_lvl_2)

([], [2325], 75)

In [113]:
n_items_before = data['item_id'].nunique()
data_train = pre_filter_items(data_train, item_features=item_features, take_n_popular=5000)
n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 89051 to 5001


In [114]:
user_item_features = get_user_item_features(data_train)
user_item_features.head(2)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

,user_id,item_id,median_sales_hour,median_weekday,mean_visits_interval,mean_check,n_stores,n_items,n_transactions,mean_n_items_basket,...,user_factor_11,user_factor_12,user_factor_13,user_factor_14,user_factor_15,user_factor_16,user_factor_17,user_factor_18,user_factor_19,user_factor_20
0,1,823721,13.0,4.0,NaN,53.097188,95,227,1328,12.875,...,-11.258902,1.792079,0.27262,1.26605,2.977834,-9.3921,-4.813135,2.912564,6.763792,5.629663
1,1,823990,15.0,6.0,NaN,53.097188,96,227,1328,12.875,...,-11.258902,1.792079,0.27262,1.26605,2.977834,-9.3921,-4.813135,2.912564,6.763792,5.629663


In [115]:
targets_test = get_targets_lvl_2(data_train, data_valid, user_item_features, N, add_to_lvl_2)

print(f'число пользователей: {targets_test.user_id.nunique()}')
print(f'среднее число покупок: {round(targets_test["target"].mean(), 4)}')

targets_test.head(2)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

число пользователей: 2272
среднее число покупок: 0.0415


,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,manufacturer_freq,...,user_factor_11,user_factor_12,user_factor_13,user_factor_14,user_factor_15,user_factor_16,user_factor_17,user_factor_18,user_factor_19,user_factor_20
0,84,999999,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84,1082185,1.0,2.0,PRODUCE,1.0,TROPICAL FRUIT,BANANAS,40 LB,1411.0,...,1.923941,-2.083421,3.708319,-1.21812,1.018669,-0.066128,0.885814,2.215996,-2.381923,1.14482


In [116]:
model_lgb = run_model_lgb(targets_test) 

[LightGBM] [Warning] Unknown parameter: class_weight
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: class_weight
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 38210, number of negative: 881593
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.081552 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12754
[LightGBM] [Info] Number of data points in the train set: 919803, number of used features: 65
[LightGBM] [Warning] Unknown parameter: class_weight
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] [bi

In [117]:
model_cb = run_model_cb(targets_test) 

0:	test: 0.8292587	best: 0.8292587 (0)	total: 332ms	remaining: 27m 38s
1000:	test: 0.8833774	best: 0.8833940 (998)	total: 4m 51s	remaining: 19m 24s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.8834198843
bestIteration = 1033

Shrink model to first 1034 iterations.


In [118]:
predictions_lgb_test = model_lgb.predict(targets_test[SELECTED_FEATURES_NAMES].fillna(0))
predictions_cb_test = model_cb.predict_proba(targets_test[SELECTED_FEATURES_NAMES_cb].fillna(0))[:, 1]

In [119]:
preds_test = pd.DataFrame(zip(predictions_lgb_test, predictions_cb_test),columns=['lgb', 'cb']).mean(axis=1).values
roc_auc_score(targets_test['target'], preds_test)

0.9059982291258616

### Обработка

In [120]:
result_test = get_results(test, targets_test, 
                           predictions_lgb_test,
                           predictions_cb_test, 
                           preds_test)
result_test.head(2)

,user_id,actual,lgb_recommendations,cb_recommendations,cb_lgb_recommendations
0,1,"[880007, 883616, 931136, 938004, 940947, 94726...","[1082185, 995242, 940947, 1029743, 1082212]","[1082185, 940947, 995242, 979707, 1082212]","[1082185, 995242, 940947, 979707, 1082212]"
1,2,"[820165, 820291, 826784, 826835, 829009, 85784...","[1106523, 1133018, 5569230, 1053690, 1075368]","[1106523, 1133018, 5569230, 1075368, 1053690]","[1106523, 1133018, 5569230, 1075368, 1053690]"


In [121]:
result_test.apply(lambda row: precision_at_k(row['lgb_recommendations'], row['actual'], 5), axis=1).mean()

0.3001591511936328

In [122]:
result_test.apply(lambda row: precision_at_k(row['cb_recommendations'], row['actual'], 5), axis=1).mean()

0.3230769230769231

In [123]:
result_test.apply(lambda row: precision_at_k(row['cb_lgb_recommendations'], row['actual'], 5), axis=1).mean()

0.3191511936339521

In [124]:
df = result_test[['user_id', 'cb_recommendations']].copy()
df.to_csv('predictions.csv', index=False)
df.head(2)

,user_id,cb_recommendations
0,1,"[1082185, 940947, 995242, 979707, 1082212]"
1,2,"[1106523, 1133018, 5569230, 1075368, 1053690]"


In [125]:
models = [model_cb, model_lgb]
for i, name in enumerate(['model_cb.pkl', 'model_lgb.pkl']):
    filename = name
    pickle.dump(models[i], open(filename, 'wb'))

In [126]:
targets_test.to_csv('targets_test.csv')